## Analysis of the Quasiparticle Method from an analytical perspective

#### Analysis of the effective one-body matrix (nn) and (pp)

Imports

In [26]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverter,HardcoreBosonsBasis
from scipy.sparse import lil_matrix
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

NSM Hamiltonian

In [27]:

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)
print(NSMHamiltonian.basis)
print(NSMHamiltonian.hamiltonian)
print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-11.93178825]
[[1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
  (0, 0)	-9.3151
  (0, 1)	0.6897999999999996
  (0, 2)	0.20328479938540372
  (0, 3)	-0.4062999999999996
  (0, 4)	-0.5082840208321855
  (0, 5)	0.18949464964372487
  (0, 6)	0.5082840208321854
  (0, 7)	0.18949464964372495
  (0, 8)	0.20328479938540372
  (0, 9)	-0

Quasiparticle Basis

In [28]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)


(6, 14)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Get the effective Hamiltonian

In [29]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T


values,psi=eigsh(hamiltonian_qq,k=1)
e0=values[0]

print(values[0])
values,psi=eigsh(hamiltonian_qq+(1/e0)*hamiltonian_qr @ hamiltonian_rq,k=1)

e1=values[0]

values,psi=eigsh(hamiltonian_qq+(1/e1)*hamiltonian_qr @ hamiltonian_rq+(1/e1**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq,k=1)

e2=values[0]
values,psi=eigsh(hamiltonian_qq+(1/e2)*hamiltonian_qr @ hamiltonian_rq+(1/e2**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq+(1/e2**3)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rr @ hamiltonian_rq,k=1)


print(values[0],np.abs((values[0]-egs[0])/egs[0]))

print(hamiltonian_qq)

-11.93178825156152
-11.931788251561521 0.0
  (0, 0)	-9.3151
  (0, 1)	0.6897999999999996
  (0, 2)	-0.4062999999999996
  (0, 3)	-0.900781889989654
  (0, 4)	-1.9314456782406446
  (0, 5)	0.6017349640876247
  (1, 0)	0.6897999999999996
  (1, 1)	-8.8615
  (1, 2)	0.8598999999999998
  (1, 3)	0.900781889989654
  (1, 4)	1.1336192497488327
  (1, 5)	-1.3995613925794368
  (2, 0)	-0.4062999999999996
  (2, 1)	0.8598999999999998
  (2, 2)	-9.145
  (2, 3)	-0.900781889989654
  (2, 4)	-0.7347060355029268
  (2, 5)	1.7984746068253425
  (3, 0)	-0.900781889989654
  (3, 1)	0.900781889989654
  (3, 2)	-0.900781889989654
  (3, 3)	-8.107099999999999
  (3, 4)	-0.7177133829043456
  (3, 5)	0.7177133829043456
  (4, 0)	-1.9314456782406446
  (4, 1)	1.1336192497488327
  (4, 2)	-0.7347060355029268
  (4, 3)	-0.7177133829043456
  (4, 4)	3.2251000000000003
  (4, 5)	0.9008999999999998
  (5, 0)	0.6017349640876247
  (5, 1)	-1.3995613925794368
  (5, 2)	1.7984746068253425
  (5, 3)	0.7177133829043456
  (5, 4)	0.9008999999999998
  (

Get the effective coupling

In [30]:
twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

print(external_potential_qq)



  (0, 0)	0.20328479938540372
  (0, 1)	-0.5082840208321855
  (0, 2)	0.18949464964372487
  (0, 3)	0.5082840208321854
  (0, 4)	0.18949464964372495
  (0, 5)	0.20328479938540372
  (0, 6)	0.48367995370807726
  (0, 7)	0.48367995370807726
  (1, 0)	0.30763695987708084
  (1, 1)	-0.10165680416643715
  (1, 2)	0.6918637014569979
  (1, 3)	0.10165680416643713
  (1, 4)	0.691863701456998
  (1, 5)	0.30763695987708073
  (1, 6)	0.09673599074161551
  (1, 7)	0.09673599074161551
  (2, 0)	0.10435216049167703
  (2, 1)	0.4066272166657483
  (2, 2)	0.502369051813273
  (2, 3)	-0.4066272166657482
  (2, 4)	0.502369051813273
  (2, 5)	0.104352160491677
  (2, 6)	-0.3869439629664618
  (2, 7)	-0.3869439629664618
  (4, 0)	0.23293633710277395
  (4, 1)	-0.15865000000000007
  (4, 2)	-0.09509586140912163
  (4, 3)	0.15865000000000004
  (4, 4)	-0.09509586140912159
  (4, 5)	0.232936337102774
  (4, 6)	0.17470000000000002
  (4, 7)	0.17470000000000002
  (5, 0)	0.23293633710277395
  (5, 1)	-0.15865000000000007
  (5, 2)	-0.0950958614

Build $H_{qq}$ using qubits

In [31]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)






one_body={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_b=np.nonzero(b)[0]
        idx_d=np.nonzero(d)[0]
        print(idx_d,idx_b)
        one_body[(idx_b[0],idx_d[0])]=hamiltonian_qq[i,j]
        one_body[(idx_b[0]+len(QPC.couples)//4,idx_d[0]+len(QPC.couples)//4)]=hamiltonian_qq[i,j]
        
print(len(one_body.keys()))

[0] [0]
[1] [0]
[2] [0]
[3] [0]
[4] [0]
[5] [0]
[0] [1]
[1] [1]
[2] [1]
[3] [1]
[4] [1]
[5] [1]
[0] [2]
[1] [2]
[2] [2]
[3] [2]
[4] [2]
[5] [2]
[0] [3]
[1] [3]
[2] [3]
[3] [3]
[4] [3]
[5] [3]
[0] [4]
[1] [4]
[2] [4]
[3] [4]
[4] [4]
[5] [4]
[0] [5]
[1] [5]
[2] [5]
[3] [5]
[4] [5]
[5] [5]
72


#### Analysis of the effective one-body matrix (np) 

Imports

In [48]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverter,HardcoreBosonsBasis
from scipy.sparse import lil_matrix
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=1
nparticles_b=1

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{18}$O'

NSM Hamiltonian

In [49]:

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)
print(NSMHamiltonian.basis)
print(NSMHamiltonian.hamiltonian)
print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-13.18138202]
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 

Quasiparticle Basis

In [50]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)


(12, 28)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


Get the effective Hamiltonian

In [51]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T


values,psi=eigsh(hamiltonian_qq,k=1)
e0=values[0]

print(values[0])
values,psi=eigsh(hamiltonian_qq+(1/e0)*hamiltonian_qr @ hamiltonian_rq,k=1)

e1=values[0]

values,psi=eigsh(hamiltonian_qq+(1/e1)*hamiltonian_qr @ hamiltonian_rq+(1/e1**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq,k=1)

e2=values[0]
values,psi=eigsh(hamiltonian_qq+(1/e2)*hamiltonian_qr @ hamiltonian_rq+(1/e2**2)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rq+(1/e2**3)*hamiltonian_qr @ hamiltonian_rr @ hamiltonian_rr @ hamiltonian_rq,k=1)


print(values[0],np.abs((values[0]-egs[0])/egs[0]))

print(hamiltonian_qq)

-12.171900000000004
-12.829526785011256 0.026693349264243153
  (0, 0)	-9.324373015873016
  (0, 1)	0.23103492063491993
  (0, 2)	-0.2881301587301585
  (0, 3)	0.11816984126984129
  (0, 4)	-0.45876507936507943
  (0, 5)	-0.009273015873015898
  (0, 6)	-0.6762161618582865
  (0, 7)	0.2245657281313674
  (0, 8)	-0.20009148833623652
  (0, 9)	0.3732125873380063
  (0, 10)	-0.2285223767496185
  (0, 11)	1.731354189904408
  (1, 0)	0.23103492063491993
  (1, 1)	-9.415725396825398
  (1, 2)	-0.10915079365079428
  (1, 3)	-0.9690507936507938
  (1, 4)	-0.5542253968253976
  (1, 5)	-0.45876507936507965
  (1, 6)	0.5858860751129027
  (1, 7)	-0.3148958148767512
  (1, 8)	0.3039641477372983
  (1, 9)	-0.15358775946623415
  (1, 10)	1.2459736331132025
  (1, 11)	-0.8296551020115345
  (2, 0)	-0.2881301587301585
  :	:
  (9, 11)	-0.6466199999999999
  (10, 0)	-0.2285223767496185
  (10, 1)	1.2459736331132025
  (10, 2)	-1.1795682823538325
  (10, 3)	0.6189063244715101
  (10, 4)	-0.15358775946623432
  (10, 5)	0.373212587338006

Get the effective coupling

In [52]:
twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

print(external_potential_qq)



  (0, 0)	-0.8929983991364584
  (0, 2)	0.6322793396075583
  (0, 4)	-0.4427846899638332
  (0, 5)	1.096283198521862
  (0, 10)	0.892998399136458
  (0, 11)	-0.6322793396075581
  (0, 13)	0.44278468996383313
  (0, 15)	-1.0962831985218615
  (1, 0)	0.08370566329509918
  (1, 2)	-0.5211067201086399
  (1, 4)	1.2129704215656378
  (1, 5)	0.2239312965819814
  (1, 10)	-0.0837056632950991
  (1, 11)	0.5211067201086399
  (1, 13)	-1.2129704215656374
  (1, 15)	-0.22393129658198155
  (2, 0)	0.18669656845047494
  (2, 2)	0.6309506098466358
  (2, 4)	-0.12858155803336288
  (2, 5)	-0.08234440795879788
  (2, 10)	-0.18669656845047483
  (2, 11)	-0.6309506098466359
  (2, 13)	0.12858155803336271
  (2, 15)	0.08234440795879784
  (3, 0)	0.0823444079587982
  :	:
  (8, 15)	-0.28108157829577807
  (9, 0)	-0.24146613733773195
  (9, 2)	-0.3675417480943267
  (9, 4)	0.2724458866852052
  (9, 5)	0.4744024744405058
  (9, 10)	0.2414661373377319
  (9, 11)	0.36754174809432677
  (9, 13)	-0.27244588668520503
  (9, 15)	-0.47440247444050

Build $H_{qq}$ using qubits

In [53]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)





for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_b=np.nonzero(b)[0]
        idx_d=np.nonzero(d)[0]
        print(idx_d,idx_b)
        one_body[(idx_b[0],idx_d[0])]=hamiltonian_qq[i,j]
        
        
print(len(one_body.keys()))

[12] [12]
[13] [12]
[14] [12]
[15] [12]
[16] [12]
[17] [12]
[18] [12]
[19] [12]
[20] [12]
[21] [12]
[22] [12]
[23] [12]
[12] [13]
[13] [13]
[14] [13]
[15] [13]
[16] [13]
[17] [13]
[18] [13]
[19] [13]
[20] [13]
[21] [13]
[22] [13]
[23] [13]
[12] [14]
[13] [14]
[14] [14]
[15] [14]
[16] [14]
[17] [14]
[18] [14]
[19] [14]
[20] [14]
[21] [14]
[22] [14]
[23] [14]
[12] [15]
[13] [15]
[14] [15]
[15] [15]
[16] [15]
[17] [15]
[18] [15]
[19] [15]
[20] [15]
[21] [15]
[22] [15]
[23] [15]
[12] [16]
[13] [16]
[14] [16]
[15] [16]
[16] [16]
[17] [16]
[18] [16]
[19] [16]
[20] [16]
[21] [16]
[22] [16]
[23] [16]
[12] [17]
[13] [17]
[14] [17]
[15] [17]
[16] [17]
[17] [17]
[18] [17]
[19] [17]
[20] [17]
[21] [17]
[22] [17]
[23] [17]
[12] [18]
[13] [18]
[14] [18]
[15] [18]
[16] [18]
[17] [18]
[18] [18]
[19] [18]
[20] [18]
[21] [18]
[22] [18]
[23] [18]
[12] [19]
[13] [19]
[14] [19]
[15] [19]
[16] [19]
[17] [19]
[18] [19]
[19] [19]
[20] [19]
[21] [19]
[22] [19]
[23] [19]
[12] [20]
[13] [20]
[14] [20]
[15] [20]


#### Proton-Neutron two body Case

Imports

In [54]:

from src.hamiltonian_utils import FermiHubbardHamiltonian
from src.nuclear_physics_utils import get_twobody_nuclearshell_model,SingleParticleState,QuadrupoleOperator,J2operator
import numpy as np
import torch
from typing import Dict
import scipy
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
from scipy.sparse.linalg import eigsh,expm_multiply
from tqdm import trange
import matplotlib.pyplot as plt
from src.utils_quasiparticle_approximation import QuasiParticlesConverter,HardcoreBosonsBasis
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$O'

NSM Hamiltonian

In [55]:

file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=2

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$Ne'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-41.39649007]
total_m= [3.27293531e-18]


Quasiparticle Basis

In [56]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)

print(QPC.rest_basis.shape)
print(QPC.couples)
print(QPC.rest_basis)

(96, 640)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 1.]]
(544, 24)
[[0, 5], [1, 4], [2, 3], [6, 7], [8, 11], [9, 10], [12, 17], [13, 16], [14, 15], [18, 19], [20, 23], [21, 22], [0, 17], [1, 16], [2, 15], [3, 14], [4, 13], [5, 12], [6, 19], [7, 18], [8, 23], [9, 22], [10, 21], [11, 20]]
[[0 1 0 ... 0 0 1]
 [1 0 1 ... 0 0 1]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Get the effective Hamiltonian

In [57]:
twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
print(twobody_qq)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

print(external_potential_qq)

  (0, 0)	-5.934974603174604
  (0, 1)	-0.10915079365079428
  (0, 2)	-0.9690507936507938
  (0, 3)	-0.5542253968253976
  (0, 4)	-0.45876507936507965
  (0, 5)	0.4680380952380956
  (0, 10)	-0.5858860751129027
  (0, 11)	0.3148958148767512
  (0, 12)	-0.3039641477372983
  (0, 13)	0.15358775946623415
  (0, 14)	-1.2459736331132025
  (0, 15)	0.8296551020115345
  (0, 16)	-0.2881301587301585
  (0, 17)	0.11816984126984129
  (0, 18)	0.46803809523809536
  (0, 19)	-0.45876507936507943
  (0, 24)	-0.009273015873015898
  (0, 25)	0.6762161618582865
  (0, 26)	-0.2245657281313674
  (0, 27)	0.20009148833623652
  (0, 28)	-0.3732125873380063
  (0, 29)	0.2285223767496185
  (0, 30)	-1.731354189904408
  (1, 0)	-0.10915079365079428
  (1, 1)	-7.013841269841272
  :	:
  (94, 94)	-6.568600000000003
  (94, 95)	-0.9184600000000009
  (95, 14)	-1.731354189904408
  (95, 15)	0.2285223767496185
  (95, 29)	0.8296551020115345
  (95, 30)	-1.2459736331132025
  (95, 43)	-0.42221262124161385
  (95, 44)	1.1795682823538325
  (95, 51)

In [59]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
#print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key
    print(idx_a,idx_b)
    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

#### twobody terms
twobody_nnpp={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_a,idx_b=np.nonzero(b)[0]
        idx_c,idx_d=np.nonzero(d)[0]

        twobody_nnpp[(idx_a,idx_b,idx_c,idx_d)]=hamiltonian_qq[i,j]-h_qq[i,j]


for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)




value,_=eigsh(h_qq,k=1)
value_hq,_=eigsh(hamiltonian_qq,k=1)
print(len(twobody_nnpp.keys()))
print(value,value_hq)

dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17), (12, 18), (12, 19), (12, 20), (12, 21), (12, 22), (12, 23), (13, 12), (13, 13), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (13, 20), (13, 21), (13, 22), (13, 23), (14, 12), (14, 13), (14, 14), (14, 15), (14, 16), (14, 17), (14, 18), (14, 19), (14, 20), (14, 21), (14, 22), (14, 23), (15, 12), (15, 13), (15, 14),

Get the effective (p-p)--(n-n) interaction

In [60]:




tot_hamiltonian=hamiltonian_qq
values,psi=eigsh(hamiltonian_qq,k=1)
e=values[0]
approximations=[]
nsteps_iteration=20
single_term = hamiltonian_rq  # Start with initial term
for i in trange(nsteps_iteration):
    if i > 0:
        single_term = hamiltonian_rr @ single_term  # Efficient update
    approximations.append(hamiltonian_qr @ single_term)  # Store result

two_body_corrections=[]
for operator in approximations:
    twobody_term={}
    for i,b in enumerate(QPC.quasiparticle_basis):
        for j,d in enumerate(QPC.quasiparticle_basis):
            idx_a,idx_b=np.nonzero(b)[0]
            idx_c,idx_d=np.nonzero(d)[0]

            twobody_term[(idx_a,idx_b,idx_c,idx_d)]=operator[i,j]#-external_potential_qq[i,j]
    two_body_corrections.append(twobody_term)


 
history_errors_exact=[]
for i in trange(nsteps_iteration):
    
    tot_hamiltonian=hamiltonian_qq
    for j in range(i):
        tot_hamiltonian=tot_hamiltonian+approximations[j]/e**(j+1)
    values,psi=eigsh(tot_hamiltonian,k=1)
    e=values[0]
    print(e)
    print(np.abs((e-egs[0])/egs[0]),'index=',i)
    history_errors_exact.append(np.abs((e-egs[0])/egs[0]))


100%|██████████| 20/20 [00:00<00:00, 223.30it/s]

-34.71498769676358
0.1614026301286271 index= 0
-36.73218977689754
0.11267381093169246 index= 1
-38.20877364058751
0.07700450995710628 index= 2
-39.18688339569876
0.05337666727294209 index= 3
-39.835549918106
0.03770706530257945 index= 4
-40.27571523052733
0.02707415145771712 index= 5
-40.580892843995294
0.019702086508902115 index= 6
-40.79656016992192
0.014492289119110149 index= 7
-40.95135316850328
0.010753010781502188 index= 8
-41.063896020644265
0.008034353911229124 index= 9
-41.146576668735335
0.006037067475926822 index= 10
-41.207843300260464
0.004557071673922217 index= 11
-41.25355943857325
0.003452723497720769 index= 12
-41.2878691424244
0.0026239164227488654 index= 13
-41.31373913914381
0.0019989842926561112 index= 14
-41.333321162666294
0.0015259484510146265 index= 15
-41.34819026407997
0.0011667609598141212 index= 16
-41.359510159306645
0.00089331035550364 index= 17
-41.368146237279234
0.0006846917514275476 index= 18
-41.37474630999704
0.000525256188020142 index= 19


#### Two-body term (nn)--(nn)

In [61]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=4
nparticles_b=0

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{20}$O'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis))

size= 24 12
[-23.93429271]
total_m= [-1.10805462e-16]


Pairing Hamiltonian one-body term

In [62]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)

print(QPC.particles2quasiparticles.shape)
print(QPC.quasiparticle_basis)

print(QPC.rest_basis.shape)
print(QPC.couples)
print(QPC.rest_basis)

external_potential_qubit=np.zeros(len(QPC.couples))

external_potential_qubit=2*SPS.energies


(15, 81)
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.

#### Two-body correction

In [63]:

hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
hamiltonian_rr=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_qr=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2restofstates.T
hamiltonian_rq=QPC.particles2restofstates @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T

twobody_qq=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_qr=QPC.particles2quasiparticles @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T
twobody_rq=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2quasiparticles.T
twobody_rr=QPC.particles2restofstates @ NSMHamiltonian.twobody_operator @ QPC.particles2restofstates.T

#print(twobody_qq)
#print(twobody_qr)

external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

#### twobody terms
twobody_nn={}
for i,b in enumerate(QPC.quasiparticle_basis):
    for j,d in enumerate(QPC.quasiparticle_basis):
        idx_a,idx_b=np.nonzero(b)[0]
        idx_c,idx_d=np.nonzero(d)[0]

        twobody_nn[(idx_a,idx_b,idx_c,idx_d)]=hamiltonian_qq[i,j]-h_qq[i,j]
        twobody_nn[(idx_a+len(QPC.couples)//4,idx_b+len(QPC.couples)//4,idx_c+len(QPC.couples)//4,idx_d+len(QPC.couples)//4)]=hamiltonian_qq[i,j]-h_qq[i,j]
        

 

h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value)

dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17), (12, 18), (12, 19), (12, 20), (12, 21), (12, 22), (12, 23), (13, 12), (13, 13), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (13, 20), (13, 21), (13, 22), (13, 23), (14, 12), (14, 13), (14, 14), (14, 15), (14, 16), (14, 17), (14, 18), (14, 19), (14, 20), (14, 21), (14, 22), (14, 23), (15, 12), (15, 13), (15, 14),

Correction for the (n-n)--(p-p) interaction

In [41]:




tot_hamiltonian=hamiltonian_qq
values,psi=eigsh(hamiltonian_qq,k=1)
e=values[0]
approximations=[]
nsteps_iteration=30
single_term = hamiltonian_rq  # Start with initial term
for i in trange(nsteps_iteration):
    if i > 0:
        single_term = hamiltonian_rr @ single_term  # Efficient update
    approximations.append(hamiltonian_qr @ single_term)  # Store result


h_qq=0.
for key,value in one_body.items():
    idx_a,idx_b=key
    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

two_body_corrections_nn=[]
for operator in approximations:
    twobody_term={}
    for i,b in enumerate(QPC.quasiparticle_basis):
        for j,d in enumerate(QPC.quasiparticle_basis):
            idx_a,idx_b=np.nonzero(b)[0]
            idx_c,idx_d=np.nonzero(d)[0]

            twobody_term[(idx_a,idx_b,idx_c,idx_d)]=operator[i,j]#-external_potential_qq[i,j]
    two_body_corrections_nn.append(twobody_term)


 
history_errors_exact=[]
for i in trange(nsteps_iteration):
    
    tot_hamiltonian=hamiltonian_qq
    for j in range(i):
        tot_hamiltonian=tot_hamiltonian+approximations[j]/e**(j+1)
    values,psi=eigsh(tot_hamiltonian,k=1)
    e=values[0]
    print(e)
    print(np.abs((e-egs[0])/egs[0]),'index=',i)
    history_errors_exact.append(np.abs((e-egs[0])/egs[0]))

100%|██████████| 30/30 [00:00<00:00, 294.15it/s]

-23.424939176438905
0.021281327927612576 index= 0
-23.66057338500436
0.011436282099960263 index= 1
-23.776440337705573
0.0065952385766171176 index= 2
-23.841536193772974
0.0038754650340764767 index= 3
-23.8793468920028
0.0022956941717962235 index= 4
-23.901575778911493
0.0013669478253038928 index= 5
-23.914738003161148
0.0008170162100236253 index= 6
-23.92257230362808
0.0004896908713382624 index= 7
-23.927253517125507
0.00029410483274151117 index= 8
-23.93005873320524
0.0001768999454309325 index= 9
-23.93174329969409
0.00010651698031123697 index= 10
-23.932756446133812
6.41866531914928e-05 index= 11
-23.933366450878253
3.8700011414513397e-05 index= 12
-23.933734017976484
2.3342670267631126e-05 index= 13
-23.933955626163623
1.4083646412587019e-05 index= 14
-23.934089289318724
8.499058778028956e-06 index= 15
-23.934169932062495
5.129719865259502e-06 index= 16
-23.934218596509588
3.0964679248053814e-06 index= 17
-23.934247968041714
1.86929432567053e-06 index= 18
-23.934265697372265
1.1285

#### The $^{24}$ Mg case

In [ ]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=4
nparticles_b=4

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{22}$Ne'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis)) 
        
        
        



size= 24 12
[-41.39649007]
total_m= [3.86247024e-18]


Build the corresponding $H_{qq}$ using qubits

In [78]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)



check the $H_{QQ}$

In [79]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T

value,_=eigsh(hamiltonian_qq,k=1)

print(value[0])

-34.714987696763565


(if you want) adding the tree level correction (diagonal)

In [80]:
#### twobody terms
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.
print(one_body.keys())
for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)
print('done the one body \n')
for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

print('done the two body \n')    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
print('done everything \n')
value,_=eigsh(h_qq,k=1)

print(value[0])
h_qq_1=h_qq.copy()

# four_body={}
# for i,b in enumerate(QPC.quasiparticle_basis):
#         idx_a1,idx_a2,idx_a3,idx_a4=np.nonzero(b)[0]
#         four_body[(idx_a1,idx_a2,idx_a3,idx_a4)]=hamiltonian_qq[i,i]-h_qq[i,i]
        
# print(four_body)
# h_qq=0.

# for key,value in one_body.items():
#     idx_a,idx_b=key

#     h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

# for key,value in twobody_nn.items():
#     idx_a,idx_b,idx_c,idx_d=key

#     h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
# for key,value in twobody_nnpp.items():
#     idx_a,idx_b,idx_c,idx_d=key

#     h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# # for key,value in four_body.items():
# #     idx_a,idx_b,idx_c,idx_d=key
# #     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)


# value,_=eigsh(h_qq,k=1)
# print(h_qq-h_qq_1)
# print(value)

dict_keys([(0, 0), (6, 6), (0, 1), (6, 7), (0, 2), (6, 8), (0, 3), (6, 9), (0, 4), (6, 10), (0, 5), (6, 11), (1, 0), (7, 6), (1, 1), (7, 7), (1, 2), (7, 8), (1, 3), (7, 9), (1, 4), (7, 10), (1, 5), (7, 11), (2, 0), (8, 6), (2, 1), (8, 7), (2, 2), (8, 8), (2, 3), (8, 9), (2, 4), (8, 10), (2, 5), (8, 11), (3, 0), (9, 6), (3, 1), (9, 7), (3, 2), (9, 8), (3, 3), (9, 9), (3, 4), (9, 10), (3, 5), (9, 11), (4, 0), (10, 6), (4, 1), (10, 7), (4, 2), (10, 8), (4, 3), (10, 9), (4, 4), (10, 10), (4, 5), (10, 11), (5, 0), (11, 6), (5, 1), (11, 7), (5, 2), (11, 8), (5, 3), (11, 9), (5, 4), (11, 10), (5, 5), (11, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17), (12, 18), (12, 19), (12, 20), (12, 21), (12, 22), (12, 23), (13, 12), (13, 13), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (13, 20), (13, 21), (13, 22), (13, 23), (14, 12), (14, 13), (14, 14), (14, 15), (14, 16), (14, 17), (14, 18), (14, 19), (14, 20), (14, 21), (14, 22), (14, 23), (15, 12), (15, 13), (15, 14),

First order

In [81]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# for key,value in four_body.items():
#     idx_a,idx_b,idx_c,idx_d=key
#     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)
value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value[0])

    


  (0, 0)	-21.637774603174606
  (0, 1)	-0.10915079365079428
  (0, 2)	-0.9690507936507938
  (0, 3)	-0.5542253968253976
  (0, 4)	-0.45876507936507965
  (0, 5)	0.4680380952380956
  (0, 10)	-0.5858860751129027
  (0, 11)	0.3148958148767512
  (0, 12)	-0.3039641477372983
  (0, 13)	0.15358775946623415
  (0, 14)	-1.2459736331132025
  (0, 15)	0.8296551020115345
  (0, 16)	-0.2881301587301585
  (0, 17)	0.11816984126984129
  (0, 18)	0.46803809523809536
  (0, 19)	-0.45876507936507943
  (0, 24)	-0.009273015873015898
  (0, 25)	0.6762161618582865
  (0, 26)	-0.2245657281313674
  (0, 27)	0.20009148833623652
  (0, 28)	-0.3732125873380063
  (0, 29)	0.2285223767496185
  (0, 30)	-1.731354189904408
  (1, 0)	-0.10915079365079428
  (1, 1)	-22.716641269841272
  (1, 2)	-1.2354015873015878
  (1, 3)	-0.9690507936507939
  (1, 4)	0.11816984126984101
  (1, 6)	0.8086333333333338
  (1, 10)	0.49555598836751885
  (1, 11)	-0.40522590162213507
  (1, 12)	0.31249341426131305
  (1, 13)	-0.6189063244715102
  (1, 14)	1.1795682823

Arbitrary order with a loop

In [82]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
print(external_potential_qq.shape)

# h_qq=0.

# for key,value in one_body.items():
#     idx_a,idx_b=key

#     h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)


    
# for key,value in twobody_nnpp.items():
#     idx_a,idx_b,idx_c,idx_d=key

#     h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
    
for r,correction in enumerate(two_body_corrections):

    delta_hamiltonian=0.    
    for j in trange(r):
        for keys,values in two_body_corrections[j].items():
            idx_a,idx_b,idx_c,idx_d=keys
            correction_term=values*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)/((e)**(j+1))
            delta_hamiltonian+=correction_term
    # for j in trange(r):
    #     for keys,values in two_body_corrections_nn[j].items():
    #         idx_a,idx_b,idx_c,idx_d=keys
    #         correction_term=values*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)/((e)**(j+1))
    #         delta_hamiltonian+=correction_term
                    
    value,_=eigsh(h_qq+delta_hamiltonian,k=1)
    e=value[0]
    print('iteration=',r,' energy=',e,'\n')



    

(15, 15)


0it [00:00, ?it/s]


iteration= 0  energy= -34.71498769676356 



100%|██████████| 1/1 [00:11<00:00, 11.06s/it]


iteration= 1  energy= -36.73218977689757 



...

iteration= 4  energy= -39.83554991810601 



100%|██████████| 5/5 [01:06<00:00, 13.24s/it]


iteration= 5  energy= -40.2757152305273 



 67%|██████▋   | 4/6 [01:05<00:32, 16.44s/it]


KeyboardInterrupt: 

#### Other Cases

In [271]:
file_name='data/usdb.nat'

SPS=SingleParticleState(file_name=file_name)

nparticles_a=2
nparticles_b=4

size_a=SPS.energies.shape[0]//2
size_b=SPS.energies.shape[0]//2

title=r'$^{22}$Ne'

# Compute the J^2 value
#J2Class=J2operator(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,single_particle_states=SPS.state_encoding,j_square_filename=file_name+'_j2',symmetries=[SPS.total_M_zero])

#Quadrupole Operator


# compute the NSM Hamiltonian
NSMHamiltonian=FermiHubbardHamiltonian(size_a=size_a,size_b=size_b,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=[SPS.total_M_zero])
print('size=',size_a+size_b,size_b)
NSMHamiltonian.get_external_potential(external_potential=SPS.energies[:size_a+size_b])
if file_name=='data/cki':
    twobody_matrix,energies=get_twobody_nuclearshell_model(file_name=file_name)

    NSMHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
else:
    NSMHamiltonian.twobody_operator=scipy.sparse.load_npz(f'data/nuclear_twobody_matrix/usdb_{nparticles_a}_{nparticles_b}.npz')
NSMHamiltonian.get_hamiltonian()

egs,psi0=NSMHamiltonian.get_spectrum(n_states=1)

print(egs)

print('total_m=',SPS.compute_m_exp_value(psi=psi0,basis=NSMHamiltonian.basis)) 
        
        
        



size= 24 12
[-57.45408791]
total_m= [4.32321381e-17]


Build the corresponding $H_{qq}$ using qubits

In [272]:
QPC=QuasiParticlesConverter()

QPC.initialize_shell(state_encoding=SPS.state_encoding)


#just for the basis
QPC.get_the_basis_matrix_transformation(basis=NSMHamiltonian.basis)



check the $H_{QQ}$

In [273]:
hamiltonian_qq=QPC.particles2quasiparticles @ NSMHamiltonian.hamiltonian @ QPC.particles2quasiparticles.T

value,_=eigsh(hamiltonian_qq,k=1)

print(value[0])

-51.352457221312235


In [274]:
print(hamiltonian_qq)

  (0, 0)	-48.05152222222223
  (0, 1)	0.8598999999999996
  (0, 2)	0.9007818899896537
  (0, 3)	1.133619249748832
  (0, 4)	-1.3995613925794361
  (0, 5)	-0.4062999999999996
  (0, 6)	-0.9007818899896537
  (0, 7)	-1.9314456782406437
  (0, 8)	0.6017349640876244
  (0, 15)	0.6897999999999996
  (0, 30)	-0.4062999999999996
  (0, 45)	-0.900781889989654
  (0, 60)	-1.9314456782406446
  (0, 75)	0.6017349640876247
  (1, 0)	0.8598999999999996
  (1, 1)	-46.40699047619047
  (1, 2)	-0.9007818899896537
  (1, 3)	-0.7347060355029265
  (1, 4)	1.7984746068253417
  (1, 5)	0.6897999999999995
  (1, 9)	-0.9007818899896537
  (1, 10)	-1.9314456782406437
  (1, 11)	0.6017349640876244
  (1, 16)	0.6897999999999996
  (1, 31)	-0.4062999999999996
  :	:
  (88, 58)	0.7177133829043456
  (88, 73)	0.9008999999999998
  (88, 77)	0.6017349640876244
  (88, 79)	-0.9007818899896537
  (88, 81)	-1.3995613925794361
  (88, 83)	0.9007818899896537
  (88, 84)	1.7984746068253417
  (88, 86)	-0.9007818899896537
  (88, 87)	0.9008999999999994
  

(if you want) adding the tree level correction (diagonal)

First order

In [275]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)

h_qq=0.

for key,value in one_body.items():
    idx_a,idx_b=key

    h_qq+=value*QubitBasis.adag_a_matrix(idx_a,idx_b)

for key,value in twobody_nn.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)
    
for key,value in twobody_nnpp.items():
    idx_a,idx_b,idx_c,idx_d=key

    h_qq+=value*QubitBasis.adag_adag_a_a_matrix(idx_a,idx_b,idx_c,idx_d)

# for key,value in four_body.items():
#     idx_a,idx_b,idx_c,idx_d=key
#     h_qq+=value*QubitBasis.four_body_matrix(idx_a,idx_b,idx_c,idx_d,idx_a,idx_b,idx_c,idx_d)
value,_=eigsh(h_qq,k=1)
print(h_qq)
print(value[0])

    


  (0, 0)	-32.348722222222236
  (0, 1)	0.8598999999999996
  (0, 2)	0.9007818899896537
  (0, 3)	1.133619249748832
  (0, 4)	-1.3995613925794361
  (0, 5)	-0.4062999999999996
  (0, 6)	-0.9007818899896537
  (0, 7)	-1.9314456782406437
  (0, 8)	0.6017349640876244
  (0, 15)	0.6897999999999996
  (0, 30)	-0.4062999999999996
  (0, 45)	-0.900781889989654
  (0, 60)	-1.9314456782406446
  (0, 75)	0.6017349640876247
  (1, 0)	0.8598999999999996
  (1, 1)	-30.704190476190497
  (1, 2)	-0.9007818899896537
  (1, 3)	-0.7347060355029265
  (1, 4)	1.7984746068253417
  (1, 5)	0.6897999999999995
  (1, 9)	-0.9007818899896537
  (1, 10)	-1.9314456782406437
  (1, 11)	0.6017349640876244
  (1, 16)	0.6897999999999996
  (1, 31)	-0.4062999999999996
  (1, 46)	-0.900781889989654
  (1, 61)	-1.9314456782406446
  (1, 76)	0.6017349640876247
  (2, 0)	0.9007818899896537
  (2, 1)	-0.9007818899896537
  (2, 2)	-29.156129365079376
  (2, 3)	-0.7177133829043454
  (2, 4)	0.7177133829043454
  (2, 6)	0.6897999999999995
  (2, 9)	-0.40629999

Second order

In [ ]:
h_qq=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        
        h_qq+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)


for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                h_qq+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l]+two_body_corrections[0][i,j,k,l]/value[0])*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                
value,_=eigsh(h_qq+external_potential_qq,k=1)

print(value)

[-53.5305729]


Third order

In [ ]:
h_qq=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        
        h_qq+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)


for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                h_qq+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l]+two_body_corrections[0][i,j,k,l]/value[0]+two_body_corrections[1][i,j,k,l]/value[0]**2)*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                
value,_=eigsh(h_qq+external_potential_qq,k=1)

print(value)

[-54.25552219]


Arbitrary order with a loop

In [ ]:
QubitBasis=HardcoreBosonsBasis(basis=QPC.quasiparticle_basis)
external_potential_qq=QPC.particles2quasiparticles @ NSMHamiltonian.external_potential @ QPC.particles2quasiparticles.T
print(external_potential_qq.shape)

hamiltonian_qq_qubit=0. 

for i in range(len(QPC.couples)): #just neutrons
        for j in range(len(QPC.couples)):
            
            hamiltonian_qq_qubit+=one_body[i,j]*QubitBasis.adag_a_matrix(i,j)
            
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                hamiltonian_qq_qubit+=(twobody_nnpp[i,j,k,l]+twobody_nn[i,j,k,l])*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                #hamiltonian_qq_qubit+=four_body[i,j,k,l]*QubitBasis.four_body_matrix(i,j,k,l,i,j,k,l)

for r,correction in enumerate(two_body_corrections[:5]):

        
    overall_twobody=np.zeros((len(QPC.couples),len(QPC.couples),len(QPC.couples),len(QPC.couples)))
    for j in range(r):
        overall_twobody+=(two_body_corrections[j])/((e)**(j+1))
    delta_hamiltonian=0.
    for i in range(len(QPC.couples)): #just neutrons
        for j in range(len(QPC.couples)):
            for k in range(len(QPC.couples)):
                for l in range(len(QPC.couples)):
                    #print(i,j,k,l)
                    #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                    delta_hamiltonian+=overall_twobody[i,j,k,l]*QubitBasis.adag_adag_a_a_matrix(i,j,k,l)
                    
    value,_=eigsh(hamiltonian_qq_qubit+delta_hamiltonian,k=1)
    e=value[0]
    print('iteration=',r,' energy=',e,'\n')

delta_fourbody=0.
for i in range(len(QPC.couples)): #just neutrons
    for j in range(len(QPC.couples)):
        for k in range(len(QPC.couples)):
            for l in range(len(QPC.couples)):
                #print(i,j,k,l)
                #print(QubitBasis.adag_adag_a_a_matrix(i,j,k,l))

                delta_fourbody+=four_body[i,j,k,l]*QubitBasis.four_body_matrix(i,j,k,l,i,j,k,l)
value,_=eigsh(hamiltonian_qq_qubit+delta_hamiltonian+delta_fourbody,k=1)
e=value[0]
print('iteration=',r,' energy=',e,'\n')

    

(225, 225)
iteration= 0  energy= -66.3824753272453 

iteration= 1  energy= -69.75675976421405 

iteration= 2  energy= -70.89029762179018 

iteration= 3  energy= -71.34028995152087 

iteration= 4  energy= -71.53242775338654 

iteration= 4  energy= -79.1134283333448 

